# Thực hiện EDA trên tập dữ liệu mua sắm tại siêu thị
Tập dữ liệu lấy từ https://www.kaggle.com/code/rajatkumar30/eda-online-retail


## 1. Import thư viện và nạp dữ liệu

In [ ]:
install.packages("ggplot2")
library(ggplot2)
install.packages("readxl")
library("readxl")
library(magrittr)
install.packages("dplyr")
library(dplyr)
install.packages("reshape2")
library(reshape2)
install.packages("plotly")
library(plotly)
install.packages("tidyr")                       
library("tidyr")

In [ ]:
df=read_excel("../input/online-retail/Online Retail.xlsx")

## Preview data

In [ ]:
head(df,10)

# Tổng quan về tập dữ liệu

## Bối cảnh

Tập dữ liệu được thu thập từ hệ thống bán lẻ trực tuyến của một công ty tại Anh Quốc, ghi nhận các giao dịch mua bán từ tháng 12 năm 2010 đến tháng 12 năm 2011. Dữ liệu phản ánh hoạt động thực tế của khách hàng tại siêu thị trực tuyến, trên nhiều quốc gia khác nhau.

## Tuyên bố

Mỗi dòng dữ liệu đại diện cho một sản phẩm được bán trong một hóa đơn, bao gồm các thông tin như:
- **CustomerID**: Mã khách hàng
- **InvoiceNo**: Số hóa đơn
- **StockCode**: Mã sản phẩm
- **Description**: Mô tả sản phẩm
- **Quantity**: Số lượng bán
- **UnitPrice**: Giá bán
- **InvoiceDate**: Ngày giao dịch
- **Country**: Quốc gia khách hàng

Ngoài ra, dữ liệu được bổ sung trường **TotalAmount** (Tổng tiền giao dịch) nhằm hỗ trợ phân tích doanh số.

## Vấn đề

Phân tích dữ liệu này nhằm mục tiêu khám phá:
- Xu hướng mua sắm của khách hàng theo thời gian, quốc gia, sản phẩm.
- Xác định sản phẩm bán chạy nhất, quốc gia có doanh số cao nhất, thời điểm mua hàng nhiều nhất.
- Phát hiện các giao dịch bất thường như trả hàng, giá trị âm, dữ liệu thiếu.
- Phân tích hành vi mua sắm để đề xuất các chiến lược marketing, kinh doanh hiệu quả hơn.

## Mục tiêu

- Thực hiện phân tích khám phá dữ liệu (EDA) để hiểu rõ cấu trúc và đặc điểm của các trường dữ liệu.
- Phân loại và nhóm các phân khúc khách hàng, sản phẩm và quốc gia dựa trên dữ liệu bán hàng.
- Đưa ra các insight hữu ích giúp doanh nghiệp tối ưu hoạt động bán hàng, tiếp thị, dự báo nhu cầu và quản lý khách.

## 2. Tiền xử lý và kiểm tra dữ liệu

In [ ]:
df=df[, c('CustomerID','InvoiceNo','StockCode','Quantity','UnitPrice','Description','InvoiceDate','Country')]
head(df)

## 3. Tóm tắt dữ liệu

In [ ]:
summary(df)

In [ ]:
str(df)

In [ ]:
nrow(df)
ncol(df)

In [ ]:

colSums(is.na(df))

## 4. Khám phá dữ liệu Country

In [ ]:
country=df$Country
country[!duplicated(country)]

## 5. Tính toán Total Amount & thêm feature Year

In [ ]:

TotalAmount = df$Quantity * df$UnitPrice
df$TotalAmount <- TotalAmount

In [ ]:
new_df = df[,c('CustomerID','InvoiceNo','StockCode','Quantity','TotalAmount','InvoiceDate','Country')]

new_df2 = df
tracemem(new_df2)==tracemem(df)

In [ ]:
head(new_df)

In [ ]:

# Nhóm các quốc gia theo Tổng số tiền bán hàng

country_price = new_df %>% 
  group_by(Country) %>% 
  summarise(Quantity = sum(Quantity))

country_price = country_price[order(-country_price$Quantity),]
country_price

In [ ]:

# Top 5 công ty có lượng mua hàng cao nhất

bp_country_top=country_price$Country[1:5]
bp_quantity_top=country_price$Quantity[1:5]
par(mar=c(2, 10, 0, 0))
barplot(height=bp_quantity_top, names=bp_country_top, col="#69b3a2", horiz=T , las=1)

In [ ]:
# 5 Công ty có số lượng mua ít nhất
bp_country_low=country_price$Country[34:38]
bp_quantity_low=country_price$Quantity[34:38]
par(mar=c(2, 10, 0, 0))
barplot(height=bp_quantity_low, names=bp_country_low, col="#69b3a2", horiz=T , las=1)

In [ ]:
# Thêm tính năng năm vào tập dữ liệu

timest <- as.POSIXct(new_df$InvoiceDate, format = "%m/%d/%Y %H:%M:%S")
timest=format(timest, format="%Y")
new_df$Year = timest

head(new_df)

## 6. Phân tích tổng số lượng và doanh thu theo năm

In [ ]:
# Biểu đồ Tổng số tiền bán hàng so với năm

year_ttlamount = new_df %>% 
  group_by(Year) %>% 
  summarise(TotalAmount = sum(TotalAmount))

bp_yearttl_Year=year_ttlamount$Year
bp_yearttl_TotalAmount=year_ttlamount$TotalAmount
par(mar=c(2, 5, 0, 0))
barplot(height=bp_yearttl_TotalAmount, names=bp_yearttl_Year, col="#69b3a2")

In [ ]:
head(new_df)

## 7. Làm sạch dữ liệu ( Loại bỏ giá trị NA )

In [ ]:
new_df=new_df %>% drop_na() 
colSums(is.na(new_df))

## 8. Phân tích số lượng khách hàng theo quốc gia

In [ ]:
# Nhóm theo số lượng khách hàng trong một quốc gia

cus_id = new_df %>% 
  group_by(Country) %>% 
  summarise(CustomerID = sum(CustomerID))

cus_id = cus_id[order(-cus_id$CustomerID),]
cus_id

In [ ]:
# 5 Quốc gia có số lượng khách hàng nhiều nhất

bp_custid_Country_top=cus_id$Country[1:5]
bp_custid_CustomerID_top=cus_id$CustomerID[1:5]
par(mar=c(2, 10, 0, 0))
barplot(height=bp_custid_CustomerID_top, names=bp_custid_Country_top, col="#69b3a2", horiz=T , las=1)

In [ ]:
# 5 Quốc gia có số lượng khách hàng ít nhất

bp_custid_Country_low=cus_id$Country[33:37]
bp_custid_CustomerID_low=cus_id$CustomerID[33:37]
par(mar=c(2, 10, 0, 0))
barplot(height=bp_custid_CustomerID_low, names=bp_custid_Country_low, col="#69b3a2", horiz=T , las=1)

In [ ]:
new_df2=new_df2 %>% drop_na()
colSums(is.na(new_df))

In [ ]:
head(new_df2)

## 9. Phân tích doanh thu trung bình từng sản phẩm

In [ ]:

# Doanh số trung bình của mỗi sản phẩm

avg_sales = new_df2 %>% 
  group_by(StockCode,Description) %>% 
  summarise(TotalAmount = mean(TotalAmount),Quantity = mean(Quantity))
avg_sales = avg_sales[order(-avg_sales$Quantity),]
avg_sales

## 10. Kiểm tra và loại bỏ giá trị bất thường (outlier):

In [ ]:


# 1. Tạo biến TotalAmount
df$TotalAmount <- df$Quantity * df$UnitPrice

# 2. Tạo new_df chỉ gồm các cột cần phân tích
new_df <- df[, c('CustomerID','InvoiceNo','StockCode','Quantity','TotalAmount','InvoiceDate','Country')]

# 3. Hàm loại bỏ outlier bằng phương pháp IQR
remove_outliers <- function(x) {
  Q1 <- quantile(x, 0.25, na.rm=TRUE)
  Q3 <- quantile(x, 0.75, na.rm=TRUE)
  IQR <- Q3 - Q1
  x >= (Q1 - 1.5*IQR) & x <= (Q3 + 1.5*IQR)
}

# 4. Áp dụng loại bỏ outlier cho Quantity và TotalAmount
out_quan <- remove_outliers(new_df$Quantity)
out_price <- remove_outliers(new_df$TotalAmount)

# 5. Tạo dataframe đã loại outlier
new_df_clean <- new_df[out_quan & out_price, ]

cat("Số dòng trước khi loại outlier:", nrow(new_df), "\n")
cat("Số dòng sau khi loại outlier:", nrow(new_df_clean), "\n")

# 6. Kiểm tra lại kết quả
summary(new_df_clean)

## 11. Phân tích trả hàng (Quantity < 0)

In [ ]:
returns <- new_df_clean %>% dplyr::filter(Quantity < 0)
cat("Số lượng giao dịch trả hàng:", nrow(returns), "\n")
cat("Các quốc gia có trả hàng:\n")
print(table(returns$Country))


## 12. Biểu đồ phân phối Quantity và TotalAmount sau khi loại outlier

In [ ]:
hist(new_df_clean$Quantity, main="Phân phối Quantity sau khi loại outlier", col="skyblue", breaks=50)
boxplot(new_df_clean$Quantity, main="Boxplot Quantity sau khi loại outlier")
hist(new_df_clean$TotalAmount, main="Phân phối TotalAmount sau khi loại outlier", col="orange", breaks=50)
boxplot(new_df_clean$TotalAmount, main="Boxplot TotalAmount sau khi loại outlier")



## 13. Doanh số theo quốc gia

In [ ]:

country_sales <- new_df_clean %>%
  dplyr::group_by(Country) %>%
  dplyr::summarise(TotalAmount = sum(TotalAmount))
ggplot(country_sales, aes(x=reorder(Country, TotalAmount), y=TotalAmount)) +
  geom_col(fill="steelblue") +
  coord_flip() +
  labs(title="Doanh so theo quoc gia", x="Quốc gia", y="Doanh số")